In [8]:
import os
import json
import openai
import time
from tqdm import tqdm
import polars as pl
from dotenv import dotenv_values

config = dotenv_values("../.env") 

In [18]:
def get_issue_reponses(title, body):
    cost = 0
    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
            {"role": "user", "content": f"Is the description of issue good? \ntitle: {title}\nbody: {body}\nAnswer with Yes/No as single word"},
        ]
    )
    description_quality = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
            {"role": "user", "content": f"Give me one sentence summary of following issue, use ten words or less: \ntitle: {title}\nbody: {body}"},
        ]
    )
    summary = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
            {"role": "user", "content": f"Suggest short label for this issue. Label should be of maximum two words: \ntitle: {title}\nbody: {body}"},
        ]
    )
    short_label = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    issue_labels = [
        "bug",
        "documentation",
        "enhancement",
        "good first issue",
        "stale"
    ]
    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
            {"role": "user", "content": f"Select label from {issue_labels}, where stale label is for issues that are too short and not clear: \ntitle: {title}\nbody: {body}\nAnswer with label name from list and nothing else"},
        ]
    )
    label = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]
    # response = openai.ChatCompletion.create(
    #     engine=config["OPENAI_ENGINE"], 
    #     messages=[
    #         {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
    #         {"role": "user", "content": f"Write first comment for: \ntitle: {title}\nbody: {body}\n"},
    #     ]
    # )
    # comment = response['choices'][0]['message']['content']

    return description_quality, summary, short_label, label, cost

In [21]:
openai.api_type = "azure"
openai.api_base = config["OPENAI_ENDPOINT"]
openai.api_version = "2023-05-15"
openai.api_key = config["OPENAI_KEY"]

data = pl.read_json("../data/issues_v1.json").filter(pl.col("number").is_not_null()).sort("number")
data = data.with_columns([
    pl.lit("").alias("description_quality"),
    pl.lit("").alias("summary"),
    pl.lit("").alias("short_label"),
    pl.lit("").alias("label"),
])

In [22]:
total_tokens = 0

In [48]:
for i in tqdm(range(len(data))):
    if data[i, "summary"] != "":
        continue
    row = data[i]
    title = row["title"]
    body = row["body"]
    passed = False
    while not passed:
        try:
            description_quality, summary, short_label, label, tokens = get_issue_reponses(title, body)
            data[i, "description_quality"] = description_quality
            data[i, "summary"] = summary
            data[i, "short_label"] = short_label
            data[i, "label"] = label
            total_tokens += tokens
            passed = True
        except (openai.error.RateLimitError, openai.error.APIConnectionError):
            time.sleep(10)
    # print(f"Total tokens: {total_tokens}")
    # print(f"Average tokens per issue: {total_tokens / (i + 1)}")
    print(f"Estimated cost: {total_tokens / 1000 * 0.028}")

 70%|██████▉   | 2058/2955 [00:02<00:00, 1017.24it/s]

Estimated cost: 25.212152
Estimated cost: 25.224556
Estimated cost: 25.236848000000002
Estimated cost: 25.249


 70%|██████▉   | 2062/2955 [00:31<00:19, 46.08it/s]  

Estimated cost: 25.261264


 70%|██████▉   | 2063/2955 [00:33<00:21, 42.47it/s]

...

Estimated cost: 36.188488


100%|█████████▉| 2954/2955 [1:53:04<00:08,  8.73s/it]

Estimated cost: 36.201032


100%|██████████| 2955/2955 [1:53:06<00:00,  2.30s/it]

Estimated cost: 36.213436


In [49]:
data.write_json("../data/issues_v1_gpt.json", row_oriented=True, pretty=True)